In [1]:
%run model.ipynb
from os.path import dirname, join, pardir

from cobra.io import read_sbml_model

from dfba import DfbaModel, ExchangeFlux, KineticVariable

from dfba.plot.plotly import *

import plotly.io as pio

'' is not a valid SBML 'SId'.


The maximum production rate of THC using the heterologous organism is 3.56 mmol/h*gDw


# **DFBA**

Here we use dynamic flux based analysis to simulate the production of THC in a batch culture.

First we have to set up the model

In [2]:
fba_model = model
fba_model.solver = "glpk"
dfba_model = DfbaModel(fba_model)

In [3]:
X = KineticVariable("Biomass")
Gluc = KineticVariable("Glucose")
THCA = KineticVariable("THCA")
Oxy = KineticVariable("Oxygen")


dfba_model.add_kinetic_variables([X, Gluc, THCA, Oxy])

mu = ExchangeFlux("DRAIN_Biomass")
v_G = ExchangeFlux("BOUNDARY_GLCe")
v_T = ExchangeFlux("THCA_e")
v_O = ExchangeFlux("BOUNDARY_O2e")

dfba_model.add_exchange_fluxes([mu, v_G, v_T, v_O])

dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("Glucose", v_G * 180.1559 * X / 1000.0)
dfba_model.add_rhs_expression("THCA", v_T * 314.45 * X / 1000.0)
dfba_model.add_rhs_expression("Oxygen", v_O * 16.0 * X / 1000.0)

dfba_model.add_exchange_flux_lb("BOUNDARY_GLCe", 10.5 * (Gluc / (0.0027 + Gluc)) , Gluc)
dfba_model.add_exchange_flux_lb("BOUNDARY_O2e", 15.0 * (Oxy / (0.024 + Oxy)), Oxy)
model.reactions.DRAIN_Biomass.lower_bound = 0.1   #This sets the growth rate to 0.1 h^-1


Now we define the initial concentrations and run the model

In [4]:
dfba_model.add_initial_conditions(
    {
        "Biomass": 1,
        "Glucose": 20.0,
        "Oxygen": 1,
        "THCA" : 0
    }
)
concentrations, trajectories = dfba_model.simulate(0.0, 12, 0.01, ["BOUNDARY_GLCe", "THCA_e", "BOUNDARY_O2e" , "DRAIN_Biomass"])

ld: warning: -pie being ignored. It is only used when linking a main executable




Final Run Statistics: 

Number of steps                    = 58
Number of residual evaluations     = 230
Number of Jacobian evaluations     = 34
Number of nonlinear iterations     = 113
Number of error test failures      = 13
Number of nonlinear conv. failures = 1
Number of root fn. evaluations     = 789


Final Run Statistics: 

Number of steps                    = 1
Number of residual evaluations     = 12
Number of Jacobian evaluations     = 3
Number of nonlinear iterations     = 6
Number of error test failures      = 2
Number of nonlinear conv. failures = 0
Number of root fn. evaluations     = 9


Final Run Statistics: 

Number of steps                    = 1
Number of residual evaluations     = 12
Number of Jacobian evaluations     = 3
Number of nonlinear iterations     = 6
Number of error test failures      = 2
Number of nonlinear conv. failures = 0
Number of root fn. evaluations     = 9


Final Run Statistics: 

Number of steps                    = 6
Number of residual evaluati

Now we can see the concentration of biomass and the metabolites during the batch

In [6]:
plot_concentrations(concentrations, x_axis_title = 'Time (h)' , left_y_axis_title = 'Biomass (g/L)' , right_y_axis_title = 'Metabolites (g/L)')


We can also see the reaction rates during the batch

In [7]:
fig = plot_trajectories(trajectories, x_axis_title = 'Time (h)' , y_axis_title = 'Flux (mmol/(g * h))' )
fig.show()